In [8]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn import cross_validation
#from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time 
from sklearn import neighbors
from sklearn.utils import shuffle
tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [9]:
mnist = fetch_mldata("MNIST original")

In [10]:
print(mnist.data.shape)

(70000, 784)


In [11]:
print(np.unique(mnist.target))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [12]:
X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X, y = shuffle(X,y)

In [13]:
#X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X_train, y_train = np.float32(X[:15000]), np.float32(y[:15000])
X_test, y_test = np.float32(X[60000:]), np.float32(y[60000:])

# PCA

In [14]:
pca = PCA(n_components=700)
X_train_pca = pca.fit_transform(X_train, y_train)
X_test_pca = pca.transform(X_test)
print('Variance covered', pca.explained_variance_ratio_.sum())
print('PCA Shape', X_train_pca.shape)

Variance covered 1.0
PCA Shape (15000, 700)


# LDA

In [15]:
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train_pca, y_train)
X_test_lda = lda.transform(X_test_pca)
print('Variance covered', lda.explained_variance_ratio_.sum())
print('PCA Shape', X_train_lda.shape)

Variance covered 1.0
PCA Shape (15000, 9)


In [16]:
start = int(round(time.time() * 1000))

In [17]:
BB = BernoulliNB()
BB.fit(X_train_lda,y_train)
end = int(round(time.time() * 1000))
print("--NN fitting finished in ", (end-start), "ms--------------")
predicted=BB.predict(X_test_lda)
expected=y_test

--NN fitting finished in  125 ms--------------


In [18]:
print(cross_validation.cross_val_score(BB, X_train_lda,y_train, cv=5))

[ 0.82217782  0.79873376  0.829       0.80860287  0.7987988 ]


In [19]:
print("Classification report for kNN classifier %s:\n%s\n"
     % (BB, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for kNN classifier BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True):
             precision    recall  f1-score   support

        0.0       0.83      0.87      0.85       931
        1.0       0.82      0.83      0.83      1176
        2.0       0.81      0.77      0.79      1016
        3.0       0.78      0.71      0.74      1009
        4.0       0.84      0.78      0.81       988
        5.0       0.69      0.65      0.67       903
        6.0       0.78      0.83      0.80       975
        7.0       0.84      0.83      0.83      1031
        8.0       0.63      0.69      0.66      1011
        9.0       0.71      0.77      0.74       960

avg / total       0.78      0.77      0.77     10000


Confusion matrix:
[[810   4  11   7   2  41  38   0  16   2]
 [  0 976  19   1   7  12  18   6 131   6]
 [ 34  24 778  26   9  11  70  25  23  16]
 [  9  26  48 714   5  58  20  26  80  23]
 [  8   9  14   3 775  11  15   7  17 129]
 [ 47  20   6 

In [20]:
print("Accuracy is:",round(metrics.accuracy_score(expected,predicted)*100,2))

Accuracy is: 77.36


In [21]:
print("Test error is:",100-round(metrics.accuracy_score(expected,predicted)*100,2))

Test error is: 22.64
